# Two-player battle version.<br>深蹲：雙人對戰版

In [2]:
%run coral_common.ipynb

In [3]:
repetition1 = squat1 = squatting1 = repetition2 = squat2 = squatting2 = 0
CONSECUTIVE_TH=3
def frame_process(frame):    
    global repetition1, squat1, squatting1, repetition2, squat2, squatting2
    
    frame_height, frame_width = frame.shape[:2]
    frame1 = (frame.transpose((1, 0, 2))[:frame_width//2]).transpose((1, 0, 2))
    frame2 = (frame.transpose((1, 0, 2))[frame_width//2:]).transpose((1, 0, 2))
    
    # calculate and display angle
    keypoints_with_scores1 = get_result_from_image_with_model(frame1, interpreter)
    keypoints_with_scores2 = get_result_from_image_with_model(frame2, interpreter)
    
    hip_angle1 = calculate_hip_joint_angle(keypoints_with_scores1)
    knee_angle1 = calculate_knee_joint_angle(keypoints_with_scores1)
    hip_angle2 = calculate_hip_joint_angle(keypoints_with_scores2)
    knee_angle2 = calculate_knee_joint_angle(keypoints_with_scores2)

    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle1 is not None and knee_angle1 is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle1<125 and knee_angle1<105:
            if squat1 == 0:
                squatting1 += 1
            else:
                squatting1 = CONSECUTIVE_TH
        elif hip_angle1>145 and knee_angle1>125:
            if squat1 == 1:
                squatting1 -= 1
            else:
                squatting1 = 0
        if squatting1 == 0 and squat1 == 1:
            repetition1 += 1
            squat1 = 0
        elif squatting1 == CONSECUTIVE_TH and squat1 == 0:
            squat1 = 1
        if squat1:
            text_color1 = (0,255,0)
            text1 = f"=  Reps: {repetition1}   "
        else:
            text_color1 = (64,240,160)
            text1 = f"+  Reps: {repetition1}   "
    else:
        # +=1 or -=1 the value of squatting based on current status, clamped within [0, CONSECUTIVE_TH]
        squatting1 = min(max((squatting1-1 if squat1 == 0 else squatting1+1), 0), CONSECUTIVE_TH)
        text1 = f"?  Reps: {repetition1}   "
        text_color1 = (128,128,128)
    frame1 = cv2.resize(frame1, (int(200/frame_height*frame_width), 400))
    putText(frame1, text1, text_color1)

    if hip_angle2 is not None and knee_angle2 is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle2<125 and knee_angle2<105:
            if squat2 == 0:
                squatting2 += 1
            else:
                squatting2 = CONSECUTIVE_TH
        elif hip_angle2>145 and knee_angle2>125:
            if squat2 == 1:
                squatting2 -= 1
            else:
                squatting2 = 0
        if squatting2 == 0 and squat2 == 1:
            repetition2 += 1
            squat2 = 0
        elif squatting2 == CONSECUTIVE_TH and squat2 == 0:
            squat2 = 1
        if squat2:
            text_color2 = (0,255,0)
            text2 = f"=  Reps: {repetition2}   "
        else:
            text_color2 = (64,240,160)
            text2 = f"+  Reps: {repetition2}   "
    else:
        # +=1 or -=1 the value of squatting based on current status, clamped within [0, CONSECUTIVE_TH]
        squatting2 = min(max((squatting2-1 if squat2 == 0 else squatting2+1), 0), CONSECUTIVE_TH)
        text2 = f"?  Reps: {repetition2} "
        text_color2 = (128,128,128)
    
    frame2 = cv2.resize(frame2, (int(200/frame_height*frame_width), 400))
    putText(frame2, text2, text_color2)

    frame = np.concatenate((frame1, np.zeros((400, 10 ,3 )) , frame2  ) , axis=1)
    
    putText(frame, f"Time: {int(time.time() - start_time)}s  FPS: "
        f"{int(1/(sum(time_process[-5:])/len(time_process[-5:]))) if time_process else 'N/A'}",
        (160,224,160) , text_position=(50, 20))
    
    return frame


    
# Run

display(stopButton)
thread = threading.Thread(target=view, args=(stopButton, 0, 1, frame_process))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None